# NOTES :
Il faut créer l'EMR avec les même versions que l'alternant et jouter le  :
- emr 6.3.0
- TensorFlow 2.4.1 (pour éviter problème keras)
- Spark 3.1.1

bootstrap S3 url : s3://p11-nicop-data/bootstrap-emr.sh

Créer le tunnel avec l'id du cluster (et P11.pem): ./scripts/setup/creer_tunnel_ssh.sh
Activer le proxy SOCKS ; ./scripts/network/gerer_proxy_mac.sh 

### 4.10.1 Démarrage de la session Spark

In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1769594848420_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des informations sur la session en cours et liens vers Spark UI</u> :

### 4.10.2 Installation des packages

Les packages nécessaires ont été installé via l'étape de **bootstrap** à l'instanciation du serveur.

### 4.10.3 Import des librairies

In [ ]:
#%pip install pandas pillow tensorflow pyarrow numpy # Pas nécessaire si bootstrap

In [2]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Vérifier que spark est disponible
print(spark)
print(spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.1.1-amzn-0

In [4]:
print(pd.__version__)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.2.5

### 4.10.4 Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [5]:
PATH = 's3a://p11-nicop-data'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'

#NP : Ajout pour PCA 
PATH_Result_PCA = PATH+'/Results_PCA'

print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result+'\nPATH_Result_PCA: '+PATH_Result_PCA)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3a://p11-nicop-data
PATH_Data:   s3a://p11-nicop-data/Test
PATH_Result: s3a://p11-nicop-data/Results
PATH_Result_PCA: s3a://p11-nicop-data/Results_PCA

### 4.10.5 Traitement des données

#### 4.10.5.1 Chargement des données

In [6]:
images = spark.read.format("binaryFile") \
    .option("pathGlobFilter", "*.jpg") \
    .option("recursiveFileLookup", "true") \
    .load(PATH_Data)

images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3a://p11-nicop-d...|2026-01-21 09:57:47| 10412|[FF D8 FF E0 00 1...|
|s3a://p11-nicop-d...|2026-01-21 09:57:47| 10224|[FF D8 FF E0 00 1...|
|s3a://p11-nicop-d...|2026-01-21 09:57:47|  9931|[FF D8 FF E0 00 1...|
|s3a://p11-nicop-d...|2026-01-21 09:57:47|  9873|[FF D8 FF E0 00 1...|
|s3a://p11-nicop-d...|2026-01-21 09:57:47|  9813|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+------------------------------------------------------+-------------+
|path                                                  |label        |
+------------------------------------------------------+-------------+
|s3a://p11-nicop-data/Test/Blackberrie 2/r0_203_100.jpg|Blackberrie 2|
|s3a://p11-nicop-data/Test/Blackberrie 2/r0_207_100.jpg|Blackberrie 2|
|s3a://p11-nicop-data/Test/Blackberrie 2/r0_199_100.jpg|Blackberrie 2|
|s3a://p11-nicop-data/Test/Blackberrie 2/r0_195_100.jpg|Blackberrie 2|
|s3a://p11-nicop-data/Test/Blackberrie 2/r0_191_100.jpg|Blackberrie 2|
+------------------------------------------------------+-------------+
only showing top 5 rows

None

#### 4.10.5.2 Préparation du modèle

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 1s 0us/step

In [10]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
sc = spark.sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [14]:
def model_fn():
    """
    Reconstruit l'architecture MobileNetV2 SANS télécharger les poids,
    puis applique les poids broadcastés (bonne pratique sur cluster).
    """
    # IMPORTANT: weights=None pour ne pas retélécharger "imagenet" sur chaque worker
    base = MobileNetV2(weights=None,
                       include_top=True,
                       input_shape=(224, 224, 3))

    for layer in base.layers:
        layer.trainable = False

    feature_extractor = Model(inputs=base.input,
                              outputs=base.layers[-2].output)

    # On injecte les poids pré-entraînés qui ont été broadcastés par le driver
    feature_extractor.set_weights(brodcast_weights.value)

    return feature_extractor

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4.10.5.3 Définition du processus de chargement des images <br/> et application de leur featurisation à travers l'utilisation de pandas UDF

In [15]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

#### 4.10.5.4 Exécutions des actions d'extractions de features

In [16]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Forcer l'évaluation avec un count() ou collect() sur un petit échantillon
print("Nombre de lignes:", features_df.count())


# Si ça fonctionne, essayez l'écriture avec un petit échantillon d'abord
features_df.limit(300).write.mode("overwrite").parquet(PATH_Result + "_test")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nombre de lignes: 42749
+--------------------+------------------+--------------------+
|                path|             label|            features|
+--------------------+------------------+--------------------+
|s3a://p11-nicop-d...|           Apple 9|[0.013158024, 0.0...|
|s3a://p11-nicop-d...|           Apple 9|[0.063711524, 0.0...|
|s3a://p11-nicop-d...|           Apple 8|[0.05239119, 0.0,...|
|s3a://p11-nicop-d...|    Apple Rotten 1|[0.0, 0.0, 0.0, 0...|
|s3a://p11-nicop-d...|           Apple 9|[0.1210034, 0.028...|
|s3a://p11-nicop-d...|Cactus fruit red 1|[0.01809258, 0.01...|
|s3a://p11-nicop-d...|Cactus fruit red 1|[0.0, 4.8492168E-...|
|s3a://p11-nicop-d...|           Apple 9|[0.0, 0.036763124...|
|s3a://p11-nicop-d...|          Apple 17|[0.0, 0.015207275...|
|s3a://p11-nicop-d...|          Apple 17|[0.0, 0.0, 0.0, 0...|
+--------------------+------------------+--------------------+

In [19]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3a://p11-nicop-data/Results

In [20]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Ajout de la PCA

In [21]:
from pyspark.ml.feature import PCA as PCAml
from pyspark.sql.functions import col
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# 1) Relire les features depuis Parquet
features_saved = spark.read.parquet(PATH_Result)

# 2) Convertir array<float> -> Vector (format attendu par Spark ML)
to_vec = udf(lambda xs: Vectors.dense(xs), VectorUDT())
df_vec = features_saved.withColumn("features_vec", to_vec(col("features")))

# 3) PCA
k = 50  
pca = PCAml(k=k, inputCol="features_vec", outputCol="pca_features")
pca_model = pca.fit(df_vec)

# voir la variance expliquée
# explainedVariance est un vecteur de taille k
ev = list(pca_model.explainedVariance)
print("Explained variance (first 10):", ev[:10])
print("Cumulative explained variance:", float(sum(ev)))

# 4) Appliquer la PCA
df_pca = pca_model.transform(df_vec)

# 5) Sortie
df_pca_out = df_pca.select("path", "label", "pca_features")

# 6) Ecriture du résultat PCA
df_pca_out.write.mode("overwrite").parquet(PATH_Result_PCA)

print("PCA written to:", PATH_Result_PCA)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Explained variance (first 10): [0.09928836429342494, 0.07368366765768797, 0.06727087602670401, 0.04037932248903533, 0.0363051063026539, 0.029903394825481225, 0.02446323377136363, 0.023474939115000166, 0.019681072433484582, 0.018856379341414216]
Cumulative explained variance: 0.7035489658869648
PCA written to: s3a://p11-nicop-data/Results_PCA

### 4.10.6 Chargement des données enregistrées et validation du résultat

In [24]:
df_spark = spark.read.parquet(PATH_Result_PCA)
df_spark.show(5, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------+--------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [26]:
df = df_spark.toPandas()
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                       pca_features
0  s3a://p11-nicop-data/Test/Apple Rotten 1/r1_10...  ...  [0.26652370659351743, -4.213250507099132, 1.51...
1   s3a://p11-nicop-data/Test/Apple 9/r2_211_100.jpg  ...  [-3.420225455701632, -10.451493865262552, 0.44...
2  s3a://p11-nicop-data/Test/Apple Rotten 1/r0_20...  ...  [-3.2388840394659493, -6.034497226865754, 6.23...
3  s3a://p11-nicop-data/Test/Apple Rotten 1/r0_19...  ...  [-1.6412101910277372, -7.848009060958037, 7.44...
4   s3a://p11-nicop-data/Test/Apple 9/r1_287_100.jpg  ...  [-3.838792857403189, -9.293911462433663, 0.546...

[5 rows x 3 columns]

In [27]:
import numpy as np

# Les pca_features sont des dictionnaires Spark ML Vector
# Il faut extraire le champ 'values' pour obtenir l'array numpy
pca_feature_dict = df.loc[0,'pca_features']

# Vérifier le type et extraire les valeurs
if isinstance(pca_feature_dict, dict):
    pca_array = np.array(pca_feature_dict['values'])
    print(f"Shape: {pca_array.shape}")
    print(f"Dimension: {pca_array.shape[0]}")
else:
    # Si c'est déjà un array (cas normal)
    print(f"Shape: {pca_feature_dict.shape}")
    # Voir la structure du premier élément
print(df.loc[0,'pca_features'])
print(type(df.loc[0,'pca_features']))

# Si c'est un dict, voir les clés
if isinstance(df.loc[0,'pca_features'], dict):
    print("Clés du dictionnaire:", df.loc[0,'pca_features'].keys())
    print("Valeurs:", df.loc[0,'pca_features']['values'])
    print("Shape des valeurs:", np.array(df.loc[0,'pca_features']['values']).shape)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Shape: (50,)
[0.26652370659351743,-4.213250507099132,1.512988594810389,11.297073922205238,-8.000269386327764,-2.329552156659796,1.2126241504415813,2.0064084381838603,-2.762435784756225,0.3698769626232579,0.9563188392070076,3.959228730304352,2.4412164342121545,2.59415810604817,-2.850261874045075,2.6623453289561527,4.838023518750753,2.4055809409780617,-1.5139105660439962,-1.7096505779596627,-0.006393493868069457,1.038117847049601,-1.1248929789621218,-0.07092766263480103,-3.2117970872827186,-3.3291652390069775,-1.8356712286568428,-1.0463837572734056,1.1888225041643359,0.5880776739352196,-3.686252710130967,0.5691075714807549,-0.11401620325288644,1.2749289044174523,-0.6545547896578212,0.8600208558177158,0.893654543673728,-1.3406055589393135,2.8603378504215313,-0.7899336651975031,1.2363504320848926,-0.030005251613472114,1.7752346049855983,0.6663788339867024,-1.279288235781963,-0.2505477578177078,-1.2409379541987202,1.9991877092669166,0.8650181507558278,0.880790292729974]
<class 'pyspark.ml.l

In [30]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(42749, 3)

In [32]:
from pyspark.sql.functions import countDistinct
print("Distinct paths Results:", spark.read.parquet(PATH_Result).select(countDistinct("path")).first()[0])
print("Distinct paths PCA:", spark.read.parquet(PATH_Result_PCA).select(countDistinct("path")).first()[0])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Distinct paths Results: 42749
Distinct paths PCA: 42749

<u>On peut également constater la présence des fichiers <br />
    au format "**parquet**" sur le **serveur S3**</u> :

![Affichage des résultats sur S3](img/S3_Results.png)

## 4.11 Suivi de l'avancement des tâches avec le Serveur d'Historique Spark

Il est possible de voir l'avancement des tâches en cours <br />
avec le **serveur d'historique Spark**.

![Accès au serveur d'historique spark](img/EMR_serveur_historique_spark_acces.png)

**Il est également possible de revenir et d'étudier les tâches <br />
qui ont été réalisé, afin de debugger, optimiser les futurs <br />
tâches à réaliser.**

<u>Lorsque la commande "**features_df.write.mode("overwrite").parquet(PATH_Result)**" <br />
était en cours, nous pouvions observer son état d'avancement</u> :

![Progression execution script](img/EMR_jupyterhub_avancement.png)

<u>Le **serveur d'historique Spark** nous permet une vision beaucoup plus précise <br />
de l'exécution des différentes tâche sur les différentes machines du cluster</u> :

![Suivi des tâches spark](img/EMR_SHSpark_01.png)

On peut également constater que notre cluster de calcul a mis <br />
un tout petit peu **moins de 8 minutes** pour traiter les **22 688 images**.

![Temps de traitement](img/EMR_SHSpark_02.png)


## 4.12 Résiliation de l'instance EMR

Notre travail est maintenant terminé. <br />
Le cluster de machines EMR est **facturé à la demande**, <br />
et nous continuons d'être facturé même lorsque <br />
les machines sont au repos.<br />
Pour **optimiser la facturation**, il nous faut <br />
maintenant **résilier le cluster**.

<u>Je réalise cette commande depuis l'interface AWS</u> :

1. Commencez par **désactiver le tunnel ssh dans FoxyProxy** pour éviter des problèmes de **timeout**.
![Désactivation de FoxyProxy](img/EMR_foxyproxy_desactivation.png)
2. Cliquez sur "**Résilier**"
![Cliquez sur Résilier](img/EMR_resiliation_01.png)
3. Confirmez la résiliation
![Confirmez la résiliation](img/EMR_resiliation_02.png)
4. La résiliation prend environ **1 minute**
![Résiliation en cours](img/EMR_resiliation_03.png)
5. La résiliation est effectuée
![Résiliation terminée](img/EMR_resiliation_04.png)

## 4.13 Cloner le serveur EMR (si besoin)

Si nous devons de nouveau exécuter notre notebook dans les mêmes conditions, <br />
il nous suffit de **cloner notre cluster** et ainsi en obtenir une copie fonctionnelle <br />
sous 15/20 minutes, le temps de son instanciation.

<u>Pour cela deux solutions</u> :
1. <u>Depuis l'interface AWS</u> :
 1. Cliquez sur "**Cloner**"
   ![Cloner un cluster](img/EMR_cloner_01.png)
 2. Dans notre cas nous ne souhaitons pas inclure d'étapes
   ![Ne pas inclure d'étapes](img/EMR_cloner_02.png)
 3. La configuration du cluster est recréée à l’identique. <br />
    On peut revenir sur les différentes étapes si on souhaite apporter des modifications<br />
    Quand tout est prêt, cliquez sur "**Créer un cluster**"
  ![Vérification/Modification/Créer un cluster](img/EMR_cloner_03.png)
2. <u>En ligne de commande</u> (avec AWS CLI d'installé et de configuré et en s'assurant <br />
   de s'attribuer les droits nécessaires sur le compte AMI utilisé)
 1. Cliquez sur "**Exporter AWS CLI**"
 ![Exporter AWS CLI](img/EMR_cloner_cli_01.png)
 2. Copier/Coller la commande **depuis un terminal**
 ![Copier Coller Commande](img/EMR_cloner_cli_02.png)

## 4.14 Arborescence du serveur S3 à la fin du projet

<u>Pour information, voici **l'arborescence complète de mon bucket S3 p8-data** à la fin du projet</u> : <br />
*Par soucis de lisibilité, je ne liste pas les 131 sous dossiers du répertoire "Test"*

1. Results/_SUCCESS
1. Results/part-00000-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00001-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00002-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00003-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00004-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00005-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00006-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00007-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00008-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00009-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00010-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00011-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00012-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00013-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00014-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00015-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00016-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00017-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00018-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00019-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00020-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00021-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00022-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Results/part-00023-2cc36f38-19ef-4d8a-a0d1-5ddb309b3894-c000.snappy.parquet
1. Test/
1. bootstrap-emr.sh
1. jupyter-s3-conf.json
1. jupyter/jovyan/.s3keep
1. jupyter/jovyan/P8_01_Notebook.ipynb
1. jupyter/jovyan/_metadata
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/file-perm.sqlite
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbconvert/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbconvert/templates/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbconvert/templates/html/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbconvert/templates/latex/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/nbsignatures.db
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.aws-editors-workspace-metadata/notebook_secret
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.ipynb_checkpoints/
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.ipynb_checkpoints/Untitled-checkpoint.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.ipynb_checkpoints/Untitled1-checkpoint.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/.ipynb_checkpoints/test3-checkpoint.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/Untitled.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/Untitled1.ipynb
1. jupyter/jovyan/e-5OTY4VKPDT21945FF6DN15E35/test3.ipynb

# 5. Conclusion

Nous avons réalisé ce projet **en deux temps** en tenant <br />
compte des contraintes qui nous ont été imposées.

Nous avons **dans un premier temps développé notre solution en local** <br />
sur une machine virtuelle dans un environnement Linux Ubuntu.

La <u>première phase</u> a consisté à **installer l'environnement de travail Spark**. <br />
**Spark** a un paramètre qui nous permet de travaillé en local et nous permet <br />
ainsi de **simuler du calcul partagé** en considérant <br />
**chaque cœur d'un processeur comme un worker indépendant**.<br />
Nous avons travaillé sur un plus **petit jeu de donnée**, l'idée était <br />
simplement de **valider le bon fonctionnement de la solution**.

Nous avons fait le choix de réaliser du **transfert learning** <br />
à partir du model **MobileNetV2**.<br />
Ce modèle a été retenu pour sa **légèreté** et sa **rapidité d'exécution** <br />
ainsi que pour la **faible dimension de son vecteur en sortie**.

Les résultats ont été enregistrés sur disque en plusieurs <br />
partitions au format "**parquet**".

<u>**La solution a parfaitement fonctionné en mode local**</u>.

La <u>deuxième phase</u> a consisté à créer un **réel cluster de calculs**. <br />
L'objectif était de pouvoir **anticiper une future augmentation de la charge de travail**.

Le meilleur choix retenu a été l'utilisation du prestataire de services **Amazon Web Services** <br />
qui nous permet de **louer à la demande de la puissance de calculs**, <br />
pour un **coût tout à fait acceptable**.<br />
Ce service se nomme **EC2** et se classe parmi les offres **Infrastructure As A Service** (IAAS).

Nous sommes allez plus loin en utilisant un service de plus <br />
haut niveau (**Plateforme As A Service** PAAS)<br />
en utilisant le service **EMR** qui nous permet d'un seul coup <br />
d'**instancier plusieurs serveur (un cluster)** sur lesquels <br />
nous avons pu demander l'installation et la configuration de plusieurs<br />
programmes et librairies nécessaires à notre projet comme **Spark**, <br />
**Hadoop**, **JupyterHub** ainsi que la librairie **TensorFlow**.

En plus d'être plus **rapide et efficace à mettre en place**, nous avons <br />
la **certitude du bon fonctionnement de la solution**, celle-ci ayant été <br />
préalablement validé par les ingénieurs d'Amazon.

Nous avons également pu installer, sans difficulté, **les packages <br />
nécessaires sur l'ensembles des machines du cluster**.

Enfin, avec très peu de modification, et plus simplement encore, <br />
nous avons pu **exécuter notre notebook comme nous l'avions fait localement**.<br />
Nous avons cette fois-ci exécuté le traitement sur **l'ensemble des images de notre dossier "Test"**.

Nous avons opté pour le service **Amazon S3** pour **stocker les données de notre projet**. <br />
S3 offre, pour un faible coût, toutes les conditions dont nous avons besoin pour stocker <br />
et exploiter de manière efficace nos données.<br />
L'espace alloué est potentiellement **illimité**, mais les coûts seront fonction de l'espace utilisé.

Il nous sera **facile de faire face à une monté de la charge de travail** en **redimensionnant** <br />
simplement notre cluster de machines (horizontalement et/ou verticalement au besoin), <br />
les coûts augmenteront en conséquence mais resteront nettement inférieurs aux coûts engendrés <br />
par l'achat de matériels ou par la location de serveurs dédiés.